# Capstone project Notebook 
###  IBM Data Science Professional Certificate 

#### Import packages

In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

#### Get data and insert it on a Dataframe

In [2]:
data_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(data_url)

soup = BeautifulSoup(page.text)

In [3]:
table_rows = soup.find('table').find('tbody').find_all('tr')

table_data = []

for row in table_rows:
    cells = row.find_all('td')
    
    if cells != []:
        postcode = cells[0].get_text()
        borough = cells[1].get_text()
        neighbourhood = cells[2].get_text()
        
        table_data.append([postcode, borough, neighbourhood.replace('\n', '')])

df = pd.DataFrame(table_data, columns=['PostalCode', 'Borough', 'Neighborhood'])

In [4]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Preprocess the data as specified

In [5]:
# Eliminate Not assigned Boroughs
df = df[df['Borough'] != 'Not assigned']

# Change Not assigned Neighborhoods to Boroughs values
for index, row in df.iterrows():
    if(row['Neighborhood'] == 'Not assigned'):
        row['Neighborhood'] = row['Borough']

In [6]:
df_new = pd.DataFrame()

df_new = pd.DataFrame(df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(','.join))
df_new.reset_index(inplace=True)

df_new.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Add the Lat&Long data

In [7]:
"""
import geocoder

lat_lng_coords = None

for row in df.iterrows():
    postal_code = df['PostalCode']
    
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    df['Latitude'] = lat_lng_coords[0]
    df['Longitude'] = lat_lng_coords[1]
    
    print('%d completed \n' % postal_code)
"""

"\nimport geocoder\n\nlat_lng_coords = None\n\nfor row in df.iterrows():\n    postal_code = df['PostalCode']\n    \n    while(lat_lng_coords is None):\n      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))\n      lat_lng_coords = g.latlng\n\n    df['Latitude'] = lat_lng_coords[0]\n    df['Longitude'] = lat_lng_coords[1]\n    \n    print('%d completed \n' % postal_code)\n"

In [8]:
# Couldn't get the latlong data using geocoder
df_postalcode = pd.read_csv('Geospatial_Coordinates.csv')
df_postalcode.head(20)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [9]:
df_new.set_index('PostalCode', inplace=True)
df_postalcode.set_index('Postal Code', inplace=True)

df = df_new.join(df_postalcode)
df.reset_index(inplace=True)

df.head()